ref: https://github.com/GINK03/pytorch-pix2pix

In [1]:
from __future__ import print_function
import argparse
import os
from math import log10
import random

import torch
import torch.nn as nn
import torch.optim as optim
from   torch.autograd         import Variable
from   torch.utils.data       import DataLoader, Dataset
import torch.backends.cudnn as cudnn
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.utils as v_utils

import cv2
import matplotlib.pyplot as plt
import numpy as np
from os.path import join
from os import listdir
import pandas as pd
import slack_notification as sn

In [2]:
%matplotlib inline

# Preprocessing

In [3]:
masks  =  pd.read_csv ('./train_ship_segmentations.csv')
masks.head()

,ImageId,EncodedPixels
0,00003e153.jpg,NaN
1,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
2,00021ddc3.jpg,101361 1 102128 3 102896 4 103663 6 104430 9 1...
3,00021ddc3.jpg,95225 2 95992 5 96760 7 97527 9 98294 9 99062 ...
4,00021ddc3.jpg,74444 4 75212 4 75980 4 76748 4 77517 3 78285 ...


In [4]:
# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
"""
ランレングス記法のデータを白黒画像に直す
"""
def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    #nan check
    if mask_rle != mask_rle:
        img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
        return img.reshape(shape).T
    
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

In [5]:
"""
trainの画像の教師画像を作る
"""
def create_mask(image_path):
    img_masks = masks.loc[masks['ImageId'] == image_path.split('/')[-1], 'EncodedPixels'].tolist()

    # Take the individual ship masks and create a single mask array for all ships
    masks_all = np.zeros((768, 768))
    for mask in img_masks:
        masks_all += rle_decode(mask)
    masks_all = np.minimum(masks_all, 1)*255
    tmp = np.array((masks_all, masks_all, masks_all), dtype=np.uint8)
    tmp = tmp.transpose(1,2,0)
    return tmp

In [6]:
"""
白黒画像をランレングス記法にする
"""
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [7]:
def convert_runlength(convert_image):
    convert_image = cv2.resize(convert_image, (768, 768))
    flatten = np.where((convert_image.T).flatten() == 1)[0]
    runlength = ""
    count = 0
    if len(flatten) == 1:
        runlength = str(flatten[0]) + " 1"
        return runlength
    for i in range(len(flatten)):
        if i == 0:
            runlength = runlength +  str(flatten[i]) + " "
            count = 1
        elif i == len(flatten)-1:
            if flatten[i] == flatten[i-1]+1:
                count += 1
                runlength = runlength + str(count)
            else:
                runlength = runlength + str(count) + " " + str(flatten[i]) + " 1"
        else:
            if flatten[i] == flatten[i-1]+1:
                count += 1
            else:
                runlength = runlength + str(count) + " " + str(flatten[i]) + " "
                count = 1
    return runlength

# Define Network

In [8]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1 or classname.find('InstanceNorm2d') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


def get_norm_layer(norm_type):
    if norm_type == 'batch':
        norm_layer = nn.BatchNorm2d
    elif norm_type == 'instance':
        norm_layer = nn.InstanceNorm2d
    else:
        print('normalization layer [%s] is not found' % norm_type)
    return norm_layer


def define_G(input_nc, output_nc, ngf, norm='batch', use_dropout=True, gpu_ids=[]):
    netG = None
    use_gpu = len(gpu_ids) > 0
    norm_layer = get_norm_layer(norm_type=norm)

    if use_gpu:
        assert(torch.cuda.is_available())

    netG = ResnetGenerator(input_nc, output_nc, ngf, norm_layer=norm_layer, use_dropout=use_dropout, n_blocks=9, gpu_ids=gpu_ids)

    if len(gpu_ids) > 0:
        netG.cuda()
    netG.apply(weights_init)
    return netG


def define_D(input_nc, ndf, norm='batch', use_sigmoid=False, gpu_ids=[]):
    netD = None
    use_gpu = len(gpu_ids) > 0
    norm_layer = get_norm_layer(norm_type=norm)

    if use_gpu:
        assert(torch.cuda.is_available())

    netD = NLayerDiscriminator(input_nc, ndf, n_layers=3, norm_layer=norm_layer, use_sigmoid=use_sigmoid, gpu_ids=gpu_ids)

    if use_gpu:
        netD.cuda()
    netD.apply(weights_init)
    return netD


def print_network(net):
    num_params = 0
    for param in net.parameters():
        num_params += param.numel()
    print(net)
    print('Total number of parameters: %d' % num_params)


# Defines the GAN loss which uses either LSGAN or the regular GAN.
class GANLoss(nn.Module):
    def __init__(self, use_lsgan=True, target_real_label=1.0, target_fake_label=0.0,
                 tensor=torch.FloatTensor):
        super(GANLoss, self).__init__()
        self.real_label = target_real_label
        self.fake_label = target_fake_label
        self.real_label_var = None
        self.fake_label_var = None
        self.Tensor = tensor
        if use_lsgan:
            self.loss = nn.MSELoss()
        else:
            self.loss = nn.BCELoss()

    def get_target_tensor(self, input, target_is_real):
        target_tensor = None
        if target_is_real:
            create_label = ((self.real_label_var is None) or
                            (self.real_label_var.numel() != input.numel()))
            if create_label:
                real_tensor = self.Tensor(input.size()).fill_(self.real_label)
                self.real_label_var = Variable(real_tensor, requires_grad=False)
            target_tensor = self.real_label_var
        else:
            create_label = ((self.fake_label_var is None) or
                            (self.fake_label_var.numel() != input.numel()))
            if create_label:
                fake_tensor = self.Tensor(input.size()).fill_(self.fake_label)
                self.fake_label_var = Variable(fake_tensor, requires_grad=False)
            target_tensor = self.fake_label_var
        return target_tensor

    def __call__(self, input, target_is_real, target_real_label=1.0, target_fake_label=0.0):
        self.real_label = target_real_label
        self.fake_label = target_fake_label
        target_tensor = self.get_target_tensor(input, target_is_real)
        return self.loss(input, target_tensor.cuda())


# Defines the generator that consists of Resnet blocks between a few
# downsampling/upsampling operations.
class ResnetGenerator(nn.Module):
    def __init__(self, input_nc, output_nc, ngf=64, norm_layer=nn.BatchNorm2d, use_dropout=True, n_blocks=6, gpu_ids=[]):
        assert(n_blocks >= 0)
        super(ResnetGenerator, self).__init__()
        self.input_nc = input_nc
        self.output_nc = output_nc
        self.ngf = ngf
        self.gpu_ids = gpu_ids

        model = [nn.Conv2d(input_nc, ngf, kernel_size=7, padding=3),
                 norm_layer(ngf, affine=True),
                 nn.ReLU(True)]

        n_downsampling = 2
        for i in range(n_downsampling):
            mult = 2**i
            model += [nn.Conv2d(ngf * mult, ngf * mult * 2, kernel_size=3,
                                stride=2, padding=1),
                      norm_layer(ngf * mult * 2, affine=True),
                      nn.ReLU(True)]

        mult = 2**n_downsampling
        for i in range(n_blocks):
            model += [ResnetBlock(ngf * mult, 'zero', norm_layer=norm_layer, use_dropout=use_dropout)]

        for i in range(n_downsampling):
            mult = 2**(n_downsampling - i)
            model += [nn.ConvTranspose2d(ngf * mult, int(ngf * mult / 2),
                                         kernel_size=3, stride=2,
                                         padding=1, output_padding=1),
                      norm_layer(int(ngf * mult / 2), affine=True),
                      nn.ReLU(True)]

        model += [nn.Conv2d(ngf, output_nc, kernel_size=7, padding=3)]
        model += [nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, input):
        if self.gpu_ids and isinstance(input.data, torch.cuda.FloatTensor):
            return nn.parallel.data_parallel(self.model, input, self.gpu_ids)
        else:
            return self.model(input)


# Define a resnet block
class ResnetBlock(nn.Module):
    def __init__(self, dim, padding_type, norm_layer, use_dropout):
        super(ResnetBlock, self).__init__()
        self.conv_block = self.build_conv_block(dim, padding_type, norm_layer, use_dropout)

    def build_conv_block(self, dim, padding_type, norm_layer, use_dropout):
        conv_block = []
        p = 0
        assert(padding_type == 'zero')
        p = 1
        
        #ここの活性化関数はLeakyReLUがいいらしい　https://qiita.com/underfitting/items/a0cbb035568dea33b2d7
        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=p),
                       norm_layer(dim, affine=True),
                       nn.LeakyReLU()]
        if use_dropout:
            conv_block += [nn.Dropout(0.5)]
        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=p),
                       norm_layer(dim, affine=True)]

        return nn.Sequential(*conv_block)

    def forward(self, x):
        out = x + self.conv_block(x)
        return out


# Defines the PatchGAN discriminator.
class NLayerDiscriminator(nn.Module):
    def __init__(self, input_nc, ndf=64, n_layers=3, norm_layer=nn.BatchNorm2d, use_sigmoid=False, gpu_ids=[]):
        super(NLayerDiscriminator, self).__init__()
        self.gpu_ids = gpu_ids

        kw = 4
        padw = int(np.ceil((kw-1)/2))
        sequence = [
            nn.Conv2d(input_nc, ndf, kernel_size=kw, stride=2, padding=padw),
            nn.LeakyReLU(0.2, True)
        ]

        nf_mult = 1
        nf_mult_prev = 1
        for n in range(1, n_layers):
            nf_mult_prev = nf_mult
            nf_mult = min(2**n, 8)
            sequence += [
                nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=2,
                          padding=padw), norm_layer(ndf * nf_mult,
                                                    affine=True), nn.LeakyReLU(0.2, True)
            ]

        nf_mult_prev = nf_mult
        nf_mult = min(2**n_layers, 8)
        sequence += [
            nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=1,
                      padding=padw), norm_layer(ndf * nf_mult,
                                                affine=True), nn.LeakyReLU(0.2, True)
        ]

        sequence += [nn.Conv2d(ndf * nf_mult, 1, kernel_size=kw, stride=1, padding=padw)]

        if use_sigmoid:
            sequence += [nn.Sigmoid()]

        self.model = nn.Sequential(*sequence)

    def forward(self, input):
        if len(self.gpu_ids)  and isinstance(input.data, torch.cuda.FloatTensor):
            return nn.parallel.data_parallel(self.model, input, self.gpu_ids)
        else:
            return self.model(input)

# Make Dataset

In [9]:
"""
train用のデータセット
"""
class AirbusDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.image_paths = os.listdir(root_dir)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = os.path.join(self.root_dir, self.image_paths[idx])
        image = cv2.imread(image_path)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        masked_img = create_mask(image_path)
        if self.transform:
            image = self.transform(image)
            masked_img = self.transform(masked_img)
        return image, masked_img

In [10]:
batch_size = 16
img_size = 256

In [11]:
train_dataset = AirbusDataset(root_dir='./train/', transform=transforms.Compose([
                                            transforms.ToPILImage(),
                                            transforms.Resize(size=img_size),
                                            transforms.ToTensor(),
                                            ]))
train_batch = data.DataLoader(train_dataset, batch_size=batch_size,
                            shuffle=True, num_workers=2)

# Generate Network

In [12]:
cudnn.benchmark = True

input_nc = 3 #入力次元
output_nc = 3#出力次元
ngf = 64
ndf = 64

cudnn.benchmark = True
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

print('===> Building model')
netG = define_G(input_nc=input_nc, output_nc=output_nc, ngf=ngf, norm='batch', use_dropout=True, gpu_ids=[0])
netD = define_D(input_nc=input_nc + output_nc, ndf=ndf, norm='batch', use_sigmoid=False, gpu_ids=[0])

criterionGAN = GANLoss()
criterionL1 = nn.L1Loss()
criterionMSE = nn.MSELoss()

# setup optimizer
lr = 0.00015
beta1 = 0.5
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))

print('---------- Networks initialized -------------')
print_network(netG)
print_network(netD)
print('-----------------------------------------------')

real_a = torch.FloatTensor(batch_size, input_nc, img_size, img_size)
real_b = torch.FloatTensor(batch_size, output_nc, img_size, img_size)

netD = netD.cuda()
netG = netG.cuda()
criterionGAN = criterionGAN.cuda()
criterionL1 = criterionL1.cuda()
criterionMSE = criterionMSE.cuda()
real_a = real_a.cuda()
real_b = real_b.cuda()

real_a = Variable(real_a)
real_b = Variable(real_b)

===> Building model
---------- Networks initialized -------------
ResnetGenerator(
  (model): Sequential(
    (0): Conv2d (3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU(inplace)
    (3): Conv2d (64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU(inplace)
    (6): Conv2d (128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (8): ReLU(inplace)
    (9): ResnetBlock(
      (conv_block): Sequential(
        (0): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
        (2): LeakyReLU(0.01)
        (3): Dropout(p=0.5)
        (4): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine

In [13]:
def train(epoch):
    losses_g = []
    losses_d = []
    checkpoint_num = 300
    for iteration, batch in enumerate(train_batch):
        print(iteration)
        # forward
        # aは衛星画像 bはマスク画像
        real_a_cpu, real_b_cpu = batch[0], batch[1]
        real_a.data.resize_(real_a_cpu.size()).copy_(real_a_cpu)
        real_b.data.resize_(real_b_cpu.size()).copy_(real_b_cpu)
        fake_b = netG(real_a)
        ############################
        # (1) Update D network: maximize log(D(x,y)) + log(1 - D(x,G(x)))
        ###########################
        optimizerD.zero_grad()
        # train with fake
        fake_ab = torch.cat((real_a, fake_b), 1)
        pred_fake = netD.forward(fake_ab.detach())
        rand_real_label = random.randint(7,12)*0.1
        rand_fake_label = random.randint(0, 3)*0.1
        loss_d_fake = criterionGAN(pred_fake, False, target_real_label=rand_real_label, target_fake_label=rand_fake_label )
        # train with real
        real_ab = torch.cat((real_a, real_b), 1)
        pred_real = netD.forward(real_ab)
        loss_d_real = criterionGAN(pred_real, True)
        # Combined loss* 0.5 
        loss_d = (loss_d_fake + loss_d_real) * 0.5
        loss_d.backward()
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(x,G(x))) + L1(y,G(x))
        ##########################
        optimizerG.zero_grad()
        # First, G(A) should fake the discriminator
        fake_ab = torch.cat((real_a, fake_b), 1)
        pred_fake = netD.forward(fake_ab)
        loss_g_gan = criterionGAN(pred_fake, True)
         # Second, G(A) = B
            
        lamb = 10
        loss_g_l1 = criterionL1(fake_b, real_b) * lamb
        loss_g = loss_g_gan + loss_g_l1 
        loss_g.backward()
        optimizerG.step()
        
        losses_g.append(loss_g.data[0])
        losses_d.append(loss_d.data[0])
        with open('./pix2pix_loss_g.txt', 'w') as f:
            f.write(str(losses_g)+"\n")
        with open('./pix2pix_loss_d.txt', 'w') as f:
            f.write(str(losses_d)+"\n")
        print("===> Epoch[{}]({}/{}): Loss_D: {:.4f} Loss_G: {:.4f}".format(
            epoch, iteration, len(train_batch), loss_d.data[0], loss_g.data[0]))
        
        if iteration % checkpoint_num == 0:
            if not os.path.exists("result_pix2pix"):
                os.mkdir("result_pix2pix")
            checkpoint(epoch, iteration)
            
            #original_img = Variable(real_a_cpu).cuda(0)
            #label_img = Variable(real_b_cpu).cuda(0).float()
            v_utils.save_image(real_a.cpu().data,"./result_{}/original_image_{}_{}.png".format(save_folder_name, epoch,iteration))
            v_utils.save_image(real_b.cpu().data,"./result_{}/label_image_{}_{}.png".format(save_folder_name, epoch,iteration))
            v_utils.save_image(fake_b.cuda(0).float().cpu().data,"./result_{}/gen_image_{}_{}.png".format(save_folder_name, epoch,iteration))
            
            plot_loss(losses_g, losses_d)
            sn.send_notification(text="Epoch[{}]({}/{}): Loss_D: {:.4f} Loss_G: {:.4f}".format(
                epoch, iteration, len(train_batch), loss_d.data[0], loss_g.data[0]))

In [14]:
def test():
    avg_psnr = 0
    for batch in testing_data_loader:
        input, target = Variable(batch[0], volatile=True), Variable(batch[1], volatile=True)
        input = input.cuda()
        target = target.cuda()

    prediction = netG(input)
    mse = criterionMSE(prediction, target)
    psnr = 10 * log10(1 / mse.data[0])
    avg_psnr += psnr
    print("===> Avg. PSNR: {:.4f} dB".format(avg_psnr / len(testing_data_loader)))

def checkpoint(epoch, iteration):
    save_folder = "model_{}".format(save_folder_name)
    net_g_model_out_path = "{}/netG_epoch{}_iter{}.pth".format(save_folder, epoch, iteration)
    net_d_model_out_path = "{}/netD_epoch{}_iter{}.pth".format(save_folder, epoch, iteration)
    torch.save(netG.state_dict(), net_g_model_out_path)
    torch.save(netD.state_dict(), net_d_model_out_path)
    
def plot_loss(losses_g, losses_d):
    losses_g_np = np.asarray(losses_g)
    losses_d_np = np.asarray(losses_d)
    batches = np.array([i for i in range(len(losses_g_np))])
    plt.figure()
    plt.plot(batches, losses_g_np, label='loss_g')
    plt.plot(batches, losses_d_np, label='loss_d')
    plt.legend()
    plt.ylabel("loss")
    plt.xlabel("batch")
    plt.savefig('./model_{}/loss.png'.format(save_folder_name))
    sn.upload_image(image='./model_{}/loss.png'.format(save_folder_name))


In [ ]:
save_folder_name = "pix2pix_0919"
if not os.path.exists("result_{}".format(save_folder_name)):
    os.mkdir("result_{}".format(save_folder_name))
else:
    print("[warning]: this file exist")
if not os.path.exists("model_{}".format(save_folder_name)):
    os.mkdir("model_{}".format(save_folder_name))
else:
    print("[warning]: this file exist")

[warning]: this file exist
[warning]: this file exist


In [ ]:
nEpochs = 10
for epoch in range(nEpochs):
    train(epoch)

0
===> Epoch[0](0/6505): Loss_D: 1.7726 Loss_G: 15.6217
1
===> Epoch[0](1/6505): Loss_D: 9.8633 Loss_G: 6.1847
2
===> Epoch[0](2/6505): Loss_D: 1.7862 Loss_G: 5.1217
3
===> Epoch[0](3/6505): Loss_D: 1.8613 Loss_G: 4.9272
4
===> Epoch[0](4/6505): Loss_D: 0.9288 Loss_G: 3.5263
5
===> Epoch[0](5/6505): Loss_D: 0.9167 Loss_G: 3.2859
6
===> Epoch[0](6/6505): Loss_D: 0.7422 Loss_G: 3.0393
7
===> Epoch[0](7/6505): Loss_D: 0.7419 Loss_G: 3.0349
8
===> Epoch[0](8/6505): Loss_D: 0.6463 Loss_G: 3.0375
9
===> Epoch[0](9/6505): Loss_D: 0.7134 Loss_G: 3.0951
10
===> Epoch[0](10/6505): Loss_D: 0.5092 Loss_G: 2.1730
11
===> Epoch[0](11/6505): Loss_D: 0.4176 Loss_G: 2.1190
12
===> Epoch[0](12/6505): Loss_D: 0.5269 Loss_G: 2.2427
13
===> Epoch[0](13/6505): Loss_D: 0.5818 Loss_G: 2.3172
14
===> Epoch[0](14/6505): Loss_D: 0.5875 Loss_G: 2.6592
15
===> Epoch[0](15/6505): Loss_D: 0.4454 Loss_G: 2.5255
16
===> Epoch[0](16/6505): Loss_D: 0.4955 Loss_G: 2.7881
17
===> Epoch[0](17/6505): Loss_D: 0.5148 Loss_G: 

===> Epoch[0](143/6505): Loss_D: 0.0693 Loss_G: 1.1838
144
===> Epoch[0](144/6505): Loss_D: 0.0704 Loss_G: 1.4289
145
===> Epoch[0](145/6505): Loss_D: 0.0978 Loss_G: 1.3088
146
===> Epoch[0](146/6505): Loss_D: 0.0627 Loss_G: 1.4529
147
===> Epoch[0](147/6505): Loss_D: 0.0827 Loss_G: 1.3908
148
===> Epoch[0](148/6505): Loss_D: 0.0473 Loss_G: 1.2437
149
===> Epoch[0](149/6505): Loss_D: 0.0642 Loss_G: 1.3113
150
===> Epoch[0](150/6505): Loss_D: 0.2463 Loss_G: 0.9789
151
===> Epoch[0](151/6505): Loss_D: 0.2035 Loss_G: 1.7634
152
===> Epoch[0](152/6505): Loss_D: 0.1936 Loss_G: 0.9787
153
===> Epoch[0](153/6505): Loss_D: 0.1173 Loss_G: 1.2245
154
===> Epoch[0](154/6505): Loss_D: 0.1231 Loss_G: 1.2038
155
===> Epoch[0](155/6505): Loss_D: 0.1687 Loss_G: 1.4958
156
===> Epoch[0](156/6505): Loss_D: 0.1958 Loss_G: 1.3125
157
===> Epoch[0](157/6505): Loss_D: 0.2247 Loss_G: 1.2367
158
===> Epoch[0](158/6505): Loss_D: 0.0821 Loss_G: 1.3534
159
===> Epoch[0](159/6505): Loss_D: 0.1215 Loss_G: 1.3210
1

===> Epoch[0](282/6505): Loss_D: 0.0629 Loss_G: 1.2073
283
===> Epoch[0](283/6505): Loss_D: 0.2500 Loss_G: 0.5399
284
===> Epoch[0](284/6505): Loss_D: 0.2383 Loss_G: 0.6729
285
===> Epoch[0](285/6505): Loss_D: 0.0924 Loss_G: 1.0507
286
===> Epoch[0](286/6505): Loss_D: 0.1266 Loss_G: 1.1226
287
===> Epoch[0](287/6505): Loss_D: 0.1018 Loss_G: 0.7282
288
===> Epoch[0](288/6505): Loss_D: 0.0721 Loss_G: 0.9830
289
===> Epoch[0](289/6505): Loss_D: 0.1150 Loss_G: 0.7924
290
===> Epoch[0](290/6505): Loss_D: 0.1102 Loss_G: 0.6704
291
===> Epoch[0](291/6505): Loss_D: 0.0898 Loss_G: 0.8842
292
===> Epoch[0](292/6505): Loss_D: 0.0939 Loss_G: 0.8370
293
===> Epoch[0](293/6505): Loss_D: 0.1912 Loss_G: 1.0276
294
===> Epoch[0](294/6505): Loss_D: 0.1050 Loss_G: 1.2240
295
===> Epoch[0](295/6505): Loss_D: 0.1636 Loss_G: 0.7952
296
===> Epoch[0](296/6505): Loss_D: 0.0721 Loss_G: 0.8006
297
===> Epoch[0](297/6505): Loss_D: 0.1680 Loss_G: 0.8318
298
===> Epoch[0](298/6505): Loss_D: 0.1300 Loss_G: 0.8758
2

===> Epoch[0](421/6505): Loss_D: 0.0720 Loss_G: 1.1082
422
===> Epoch[0](422/6505): Loss_D: 0.0801 Loss_G: 1.1039
423
===> Epoch[0](423/6505): Loss_D: 0.0676 Loss_G: 0.9029
424
===> Epoch[0](424/6505): Loss_D: 0.0347 Loss_G: 0.9072
425
===> Epoch[0](425/6505): Loss_D: 0.0333 Loss_G: 1.1032
426
===> Epoch[0](426/6505): Loss_D: 0.0661 Loss_G: 0.9754
427
===> Epoch[0](427/6505): Loss_D: 0.0463 Loss_G: 1.1535
428
===> Epoch[0](428/6505): Loss_D: 0.0500 Loss_G: 1.1905
429
===> Epoch[0](429/6505): Loss_D: 0.0688 Loss_G: 0.8179
430
===> Epoch[0](430/6505): Loss_D: 0.0706 Loss_G: 0.9169
431
===> Epoch[0](431/6505): Loss_D: 0.0568 Loss_G: 1.1585
432
===> Epoch[0](432/6505): Loss_D: 0.0807 Loss_G: 1.2551
433
===> Epoch[0](433/6505): Loss_D: 0.0607 Loss_G: 0.9510
434
===> Epoch[0](434/6505): Loss_D: 0.0432 Loss_G: 1.1188
435
===> Epoch[0](435/6505): Loss_D: 0.1152 Loss_G: 0.7146
436
===> Epoch[0](436/6505): Loss_D: 0.0550 Loss_G: 0.8363
437
===> Epoch[0](437/6505): Loss_D: 0.0204 Loss_G: 1.0321
4

===> Epoch[0](560/6505): Loss_D: 0.0250 Loss_G: 1.0757
561
===> Epoch[0](561/6505): Loss_D: 0.1256 Loss_G: 0.7004
562
===> Epoch[0](562/6505): Loss_D: 0.1787 Loss_G: 0.8924
563
===> Epoch[0](563/6505): Loss_D: 0.0461 Loss_G: 0.8139
564
===> Epoch[0](564/6505): Loss_D: 0.1212 Loss_G: 0.9174
565
===> Epoch[0](565/6505): Loss_D: 0.0338 Loss_G: 1.3817
566
===> Epoch[0](566/6505): Loss_D: 0.0724 Loss_G: 1.0238
567
===> Epoch[0](567/6505): Loss_D: 0.0437 Loss_G: 1.0979
568
===> Epoch[0](568/6505): Loss_D: 0.0663 Loss_G: 1.1314
569
===> Epoch[0](569/6505): Loss_D: 0.0231 Loss_G: 0.8172
570
===> Epoch[0](570/6505): Loss_D: 0.0382 Loss_G: 0.9955
571
===> Epoch[0](571/6505): Loss_D: 0.0187 Loss_G: 1.2481
572
===> Epoch[0](572/6505): Loss_D: 0.0863 Loss_G: 1.1566
573
===> Epoch[0](573/6505): Loss_D: 0.1709 Loss_G: 1.5832
574
===> Epoch[0](574/6505): Loss_D: 0.0613 Loss_G: 0.7835
575
===> Epoch[0](575/6505): Loss_D: 0.0709 Loss_G: 1.2008
576
===> Epoch[0](576/6505): Loss_D: 0.1778 Loss_G: 0.5895
5

...

===> Epoch[2](883/6505): Loss_D: 0.0662 Loss_G: 0.4966
884
===> Epoch[2](884/6505): Loss_D: 0.0529 Loss_G: 0.5369
885
===> Epoch[2](885/6505): Loss_D: 0.1662 Loss_G: 0.5983
886
===> Epoch[2](886/6505): Loss_D: 0.0547 Loss_G: 0.5611
887
===> Epoch[2](887/6505): Loss_D: 0.3098 Loss_G: 0.3180
888
===> Epoch[2](888/6505): Loss_D: 0.0896 Loss_G: 0.3372
889
===> Epoch[2](889/6505): Loss_D: 0.1672 Loss_G: 0.2425
890
===> Epoch[2](890/6505): Loss_D: 0.0695 Loss_G: 0.6905
891
===> Epoch[2](891/6505): Loss_D: 0.0653 Loss_G: 0.5463
892
===> Epoch[2](892/6505): Loss_D: 0.1984 Loss_G: 0.2878
893
===> Epoch[2](893/6505): Loss_D: 0.0681 Loss_G: 0.5712
894
===> Epoch[2](894/6505): Loss_D: 0.2019 Loss_G: 0.2926
895
===> Epoch[2](895/6505): Loss_D: 0.3301 Loss_G: 0.2904
896
===> Epoch[2](896/6505): Loss_D: 0.0208 Loss_G: 0.5995
897
===> Epoch[2](897/6505): Loss_D: 0.2016 Loss_G: 0.2831
898
===> Epoch[2](898/6505): Loss_D: 0.1547 Loss_G: 0.4100
899
===> Epoch[2](899/6505): Loss_D: 0.2321 Loss_G: 0.3589
9

===> Epoch[2](1022/6505): Loss_D: 0.1712 Loss_G: 0.6419
1023
===> Epoch[2](1023/6505): Loss_D: 0.1169 Loss_G: 0.4906
1024
===> Epoch[2](1024/6505): Loss_D: 0.0848 Loss_G: 0.4666
1025
===> Epoch[2](1025/6505): Loss_D: 0.0803 Loss_G: 0.5088
1026
===> Epoch[2](1026/6505): Loss_D: 0.0206 Loss_G: 0.7139
1027
===> Epoch[2](1027/6505): Loss_D: 0.0277 Loss_G: 0.7437
1028
===> Epoch[2](1028/6505): Loss_D: 0.0815 Loss_G: 0.4983
1029
===> Epoch[2](1029/6505): Loss_D: 0.1128 Loss_G: 0.3917
1030
===> Epoch[2](1030/6505): Loss_D: 0.0775 Loss_G: 0.4910
1031
===> Epoch[2](1031/6505): Loss_D: 0.0183 Loss_G: 0.7011
1032
===> Epoch[2](1032/6505): Loss_D: 0.0317 Loss_G: 0.7140
1033
===> Epoch[2](1033/6505): Loss_D: 0.0987 Loss_G: 0.4929
1034
===> Epoch[2](1034/6505): Loss_D: 0.1258 Loss_G: 0.5146
1035
===> Epoch[2](1035/6505): Loss_D: 0.3966 Loss_G: 0.3134
1036
===> Epoch[2](1036/6505): Loss_D: 0.2944 Loss_G: 0.2261
1037
===> Epoch[2](1037/6505): Loss_D: 0.3350 Loss_G: 0.3101
1038
===> Epoch[2](1038/6505)

===> Epoch[2](1157/6505): Loss_D: 0.4173 Loss_G: 0.1931
1158
===> Epoch[2](1158/6505): Loss_D: 0.1879 Loss_G: 0.4245
1159
===> Epoch[2](1159/6505): Loss_D: 0.1396 Loss_G: 0.3712
1160
===> Epoch[2](1160/6505): Loss_D: 0.1578 Loss_G: 0.3330
1161
===> Epoch[2](1161/6505): Loss_D: 0.3251 Loss_G: 0.2120
1162
===> Epoch[2](1162/6505): Loss_D: 0.1057 Loss_G: 0.3811
1163
===> Epoch[2](1163/6505): Loss_D: 0.1334 Loss_G: 0.5417
1164
===> Epoch[2](1164/6505): Loss_D: 0.1358 Loss_G: 0.3659
1165
===> Epoch[2](1165/6505): Loss_D: 0.1271 Loss_G: 0.3489
1166
===> Epoch[2](1166/6505): Loss_D: 0.2219 Loss_G: 0.2082
1167
===> Epoch[2](1167/6505): Loss_D: 0.0339 Loss_G: 0.7015
1168
===> Epoch[2](1168/6505): Loss_D: 0.1649 Loss_G: 0.4986
1169
===> Epoch[2](1169/6505): Loss_D: 0.2024 Loss_G: 0.4336
1170
===> Epoch[2](1170/6505): Loss_D: 0.0968 Loss_G: 0.5371
1171
===> Epoch[2](1171/6505): Loss_D: 0.1159 Loss_G: 0.4878
1172
===> Epoch[2](1172/6505): Loss_D: 0.1001 Loss_G: 0.6715
1173
===> Epoch[2](1173/6505)

===> Epoch[2](1292/6505): Loss_D: 0.1668 Loss_G: 0.2671
1293
===> Epoch[2](1293/6505): Loss_D: 0.1616 Loss_G: 0.3072
1294
===> Epoch[2](1294/6505): Loss_D: 0.1932 Loss_G: 0.2219
1295
===> Epoch[2](1295/6505): Loss_D: 0.1827 Loss_G: 0.2365
1296
===> Epoch[2](1296/6505): Loss_D: 0.1480 Loss_G: 0.2786
1297
===> Epoch[2](1297/6505): Loss_D: 0.2458 Loss_G: 0.2220
1298
===> Epoch[2](1298/6505): Loss_D: 0.1796 Loss_G: 0.3208
1299
===> Epoch[2](1299/6505): Loss_D: 0.0978 Loss_G: 0.4947
1300
===> Epoch[2](1300/6505): Loss_D: 0.2457 Loss_G: 0.2153
1301
===> Epoch[2](1301/6505): Loss_D: 0.1153 Loss_G: 0.3975
1302
===> Epoch[2](1302/6505): Loss_D: 0.1142 Loss_G: 0.4165
1303
===> Epoch[2](1303/6505): Loss_D: 0.0779 Loss_G: 0.4374
1304
===> Epoch[2](1304/6505): Loss_D: 0.1766 Loss_G: 0.3183
1305
===> Epoch[2](1305/6505): Loss_D: 0.1718 Loss_G: 0.2807
1306
===> Epoch[2](1306/6505): Loss_D: 0.1603 Loss_G: 0.2478
1307
===> Epoch[2](1307/6505): Loss_D: 0.1409 Loss_G: 0.3876
1308
===> Epoch[2](1308/6505)

===> Epoch[2](1427/6505): Loss_D: 0.1724 Loss_G: 0.3012
1428
===> Epoch[2](1428/6505): Loss_D: 0.1433 Loss_G: 0.2663
1429
===> Epoch[2](1429/6505): Loss_D: 0.1270 Loss_G: 0.2837
1430
===> Epoch[2](1430/6505): Loss_D: 0.1516 Loss_G: 0.3517
1431
===> Epoch[2](1431/6505): Loss_D: 0.1282 Loss_G: 0.3245
1432
===> Epoch[2](1432/6505): Loss_D: 0.1172 Loss_G: 0.3422
1433
===> Epoch[2](1433/6505): Loss_D: 0.1378 Loss_G: 0.2776
1434
===> Epoch[2](1434/6505): Loss_D: 0.2297 Loss_G: 0.3441
1435
===> Epoch[2](1435/6505): Loss_D: 0.1283 Loss_G: 0.4772
1436
===> Epoch[2](1436/6505): Loss_D: 0.1472 Loss_G: 0.3886
1437
===> Epoch[2](1437/6505): Loss_D: 0.2193 Loss_G: 0.1889
1438
===> Epoch[2](1438/6505): Loss_D: 0.1680 Loss_G: 0.3021
1439
===> Epoch[2](1439/6505): Loss_D: 0.1771 Loss_G: 0.2719
1440
===> Epoch[2](1440/6505): Loss_D: 0.1692 Loss_G: 0.3097
1441
===> Epoch[2](1441/6505): Loss_D: 0.1203 Loss_G: 0.4138
1442
===> Epoch[2](1442/6505): Loss_D: 0.1785 Loss_G: 0.3616
1443
===> Epoch[2](1443/6505)